In [1]:
from utils import time_op
import impmeas as imp

In [2]:
# tests the model counting capabilities of SharpSAT-TD and Buddy

n,m,k = 8, 10, 7
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx}" for idx in range(1, n+1)]

solver = imp.formulas.GPMC()
satcount_gpmc = time_op(solver.satcount)(cnf)
print(f"GPMC satcount = {satcount_gpmc}")

with imp.formulas.BuddyContext(vars) as ctx:
    imp.formulas.set_buddy_context(ctx)
    f = time_op(imp.BuddyNode.parse)(formula)
    satcount_bdd = time_op(f.expectation)() * 2**n
    print(f"BDD satcount = {satcount_bdd}")
    del f

[00018.3890 ms / 0.0184 s / 0.0003 min] satcount
GPMC satcount = 240
[00077.8689 ms / 0.0779 s / 0.0013 min] parse
[00000.1731 ms / 0.0002 s / 0.0000 min] expectation
BDD satcount = 240.0


In [3]:
n,clauses_to_variables,k = 17, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [4]:
import impmeas.bdds as bdd 

with imp.formulas.BuddyContext(vars) as ctx:
    imp.formulas.set_buddy_context(ctx)
    x = "x1"
    f = time_op(imp.BuddyNode.parse)(formula)
    print(f"bdd size = {f.nodecount}")
    blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
    infl = time_op(imp.influence)(f, x)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame")
    del f

[01236.0365 ms / 1.2360 s / 0.0206 min] parse
bdd size = 1999
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7f54fea63100> ===
k=0 d result=0.1674 max increase possible=0.4163 
k=1 d result=0.1778 max increase possible=0.1590 
k=2 d result=0.1242 max increase possible=0.0261 
k=3 d result=0.0251 max increase possible=0.0008 
k=4 d result=0.0008 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.4953, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[06806.9801 ms / 6.8070 s / 0.1134 min] blame
[00000.3688 ms / 0.0004 s / 0.0000 min] influence
influence 0.16743, 0.49534 <= blame


In [5]:
solver = imp.formulas.GPMC(cs=20000)
imp.formulas.set_pmc_solver(solver)
f = time_op(imp.Formula.parse)(formula)
x = "x1"
infl = (time_op)(imp.influence)(f, x)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")

[01393.5242 ms / 1.3935 s / 0.0232 min] parse
[00264.7078 ms / 0.2647 s / 0.0044 min] influence
=== COMPUTING BLAME for x1 in Formula with size 3192 ===
k=0 size of cnf: 3422 d result=0.1674 max increase possible=0.4163 
k=1 size of cnf: 3481 d result=0.1778 max increase possible=0.1590 
k=2 size of cnf: 3536 d result=0.1242 max increase possible=0.0261 
k=3 size of cnf: 3587 d result=0.0251 max increase possible=0.0008 
k=4 size of cnf: 3634 d result=0.0008 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.4953, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[15996.3989 ms / 15.9964 s / 0.2666 min] blame
influence 0.16743, 0.49534 <= blame


In [6]:

n,clauses_to_variables,k = 10, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]
x = "x1"

f = time_op(imp.Table.parse)(formula)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=False)
print(f"{blame:.5f} <= blame")
infl = time_op(imp.influence)(f, x)
print(f"influence = {infl:.5f}")
hammer = time_op(lambda f,x: imp.banzhaf(imp.hkr_cgm(f),x))(f, x)
print(f"hammer = {hammer:.5f}")
omega = time_op(imp.dominating_cgm)(f)
upsilon = time_op(imp.rectifying_cgm)(f)

print("="*100)

with imp.formulas.BuddyContext(vars) as ctx:
    imp.formulas.set_buddy_context(ctx)
    f = time_op(imp.BuddyNode.parse)(formula)
    print(f"bdd size = {f.nodecount}")
    blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
    infl = time_op(imp.influence)(f, x)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame")


SyntaxError: invalid syntax (3067685722.py, line 17)